<a href="https://colab.research.google.com/github/kavyajeetbora/geemap/blob/master/end_to_end_earth_engine/Module_02_Earth_Engine_Intermediate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Module 2 builds on the basic Earth Engine skills you have gained. This model introduces the parallel programming concepts using Map/Reduce - which is key in effectively using Earth Engine for analyzing large volumes of data. You will learn how to use the Earth Engine API for calculating various spectral indices, do cloud masking and then use map/reduce to do apply these computations to collections of imagery. You will also learn how to take long time-series of data and create charts.



[<img src='https://courses.spatialthoughts.com/images/end_to_end_gee/map_reduce.png' height=100/>](https://docs.google.com/presentation/d/10qOyxhubkwnsAVjniW54ETgwUHq3DXYKo3HGb6Gemi0/edit)

In [5]:
import ee
import geemap

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Earth Engine Objects

This script introduces the basics of the Earth Engine API. When programming in Earth Engine, you must use the Earth Engine API so that your computations can use the Google Earth Engine servers

In [6]:
myList = list(range(0,10))

def func(number):
    return number+1

print(func(3))

4


In [7]:
def func2(number):
    return ee.Number(number).add(1)

print(func2(3))

ee.Number({
  "functionInvocationValue": {
    "functionName": "Number.add",
    "arguments": {
      "left": {
        "constantValue": 3
      },
      "right": {
        "constantValue": 1
      }
    }
  }
})


In [10]:
print(list(map(func, myList)))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
